# Copyright 2026 Cognite AS

## Import the Libraries and Modules

In [1]:
import sys
from pathlib import Path

utils = str(Path("../utils").resolve())
if utils not in sys.path:
    sys.path.append(utils)

from cognite_auth import interactive_client

Importing everything that is needed: 

In [2]:
from cognite.client.data_classes.data_modeling.cdm.v1 import (
    CogniteAsset,
    CogniteAssetApply,
    CogniteTimeSeriesApply,
)
from cognite.client.data_classes.data_modeling import (
    DirectRelationReference,
    NodeId,
    ViewId,
    NodeApply,
    NodeOrEdgeData,
)
from cognite.client.data_classes.filters import Prefix

## Create the Cognite Client

In [3]:
client = interactive_client()

#### Add your prefix to make sure that the data you create is unique.

You can for example use your *name* as a prefix, or some random letters - or something else, like `golden_duck`.

In [ ]:
prefix = "my_first_name" # REMEBER TO CHANGE THIS!

assert prefix != "my_first_name", "Please change the prefix to avoid messing with other people's data!"

### CogniteAssets

##### Here we are creating 2 seperate root CogniteAssets

In [5]:
assets = [
    CogniteAssetApply(
        space="playground",
        external_id=f"{prefix}_asset1",
        name="asset1",
        parent=None,
    ),
    CogniteAssetApply(
        space="playground",
        external_id=f"{prefix}_asset2",
        name="asset2",
        parent=None,
    ),
]
client.data_modeling.instances.apply(nodes=assets)

InstancesApplyResult(nodes=[<NodeApplyResult(space='playground', external_id='delete_me_asset1', version=1) at 0x10f44af00>, <NodeApplyResult(space='playground', external_id='delete_me_asset2', version=1) at 0x11130c050>], edges=[])

##### Although we prefer to used the typed classes like CogniteAssetApply, you can also use the more generic NodeApply and NodeOrEdgeData

In [ ]:
asset_view = CogniteAsset.get_source()
# ...or: ViewId("cdf_cdm", "CogniteAsset", "v1")

NodeApply(
    space="playground",
    external_id=f"{prefix}_asset1",
    sources=[
        NodeOrEdgeData(
            source=asset_view,
            properties={"name": "asset1", "parent": None},
        )
    ],
);

##### In this learning envirnoment, everyone adds the CogniteAssets with the same name into the same environment. If we wanted to retrieve the CogniteAssets just created, we need to filter on the prefix we used, to not see all the similar-looking assets created by other course-takers. To achieve this, we use a Prefix filter:

In [11]:
from cognite.client.data_classes.filters import Prefix

# externalId is a base node property. → We can reference it by using ("node", "externalId") in filters:
flt = Prefix(
    property=("node", "externalId"),
    value=prefix,
)

res = client.data_modeling.instances.list(
    sources=asset_view,
    filter=flt,
    limit=None,
)
res

,space,external_id,version,last_updated_time,created_time,instance_type,path,root,pathLastUpdatedTime,name
0,playground,delete_me_asset1,2,2026-02-23 23:27:48.043,2026-02-23 23:23:07.035,node,"[{'space': 'playground', 'externalId': 'delete...","{'space': 'playground', 'externalId': 'delete_...",2026-02-23T23:27:48.043434+00:00,asset1
1,playground,delete_me_asset2,2,2026-02-23 23:27:48.043,2026-02-23 23:23:07.035,node,"[{'space': 'playground', 'externalId': 'delete...","{'space': 'playground', 'externalId': 'delete_...",2026-02-23T23:27:48.043434+00:00,asset2


### CogniteAssets Hierachy

##### CogniteAssets can also be organized into a hierarchy. Lets look at how we can create such a tree structure:

In [12]:
root = CogniteAssetApply(
    space="playground",
    external_id=f"{prefix}_root",
    name=prefix + " Root",
    parent=None,
)
child1 = CogniteAssetApply(
    space="playground",
    external_id=f"{prefix}_child1",
    name="child1",
    parent=DirectRelationReference("playground", f"{prefix}_root"),
)
child2 = CogniteAssetApply(
    space="playground",
    external_id=f"{prefix}_child2",
    name="child2",
    parent=DirectRelationReference("playground", f"{prefix}_root"),
)
child_of_child1 = CogniteAssetApply(
    space="playground",
    external_id=f"{prefix}_child_of_child1",
    name="child_of_child1",
    parent=DirectRelationReference("playground", f"{prefix}_child1"),
)

client.data_modeling.instances.apply(nodes=[root, child1, child2, child_of_child1])

InstancesApplyResult(nodes=[<NodeApplyResult(space='playground', external_id='delete_me_root', version=1) at 0x11290e1e0>, <NodeApplyResult(space='playground', external_id='delete_me_child1', version=1) at 0x11290e2d0>, <NodeApplyResult(space='playground', external_id='delete_me_child2', version=1) at 0x11290e1b0>, <NodeApplyResult(space='playground', external_id='delete_me_child_of_child1', version=1) at 0x11290da30>], edges=[<EdgeApplyResult(space='playground', external_id='h66W5VV9lVVoyOhxUY3aWldWLSfc5wKTYhzDbN8p__c=', version=1) at 0x11290d910>, <EdgeApplyResult(space='playground', external_id='IJmTYwZ-KJ2WWQ9EeZ8MYnPMlIdXyBRNIqfpomAT2F8=', version=1) at 0x11290ddf0>, <EdgeApplyResult(space='playground', external_id='YfoybXr8MtRPAdspe2SlKwvFD0uVr1MywPoSJKSLS7k=', version=1) at 0x11290e270>])

#### Lets look at the CogniteAsset Hierarchy that was created

**Important**: When you upsert assets, your changes are potentially affecting millions of existing instances! Asset hierarchies can become *very* large(!). For this reason, we have a separate part of the API that monitors for asset changes and makes such cascading updates automatically. This is called "path materializer". For such a small tree as you created here, it should finish almost immediately.

It is responsible for maintaining the following properties:
- `path`: The path to the asset in the hierarchy (list of direct relations)
- `root`: A direct relation to the root asset

Example: For a hierarchy starting with root A: A -> B -> C, the path of B will be [A, B] and the root will be A.

In [20]:
# 1) Get the "child1" CogniteAsset node, which itself has a child:
child1_asset = client.data_modeling.instances.retrieve_nodes(
    NodeId("playground", f"{prefix}_child1"),
    node_cls=CogniteAsset,
)
# Check that the path attribute has been computed (read the section above)
assert child1_asset.path, "Wait a few seconds for the path attribute to be computed"

child1_asset.path

[DirectRelationReference(space='playground', external_id='delete_me_root'),
 DirectRelationReference(space='playground', external_id='delete_me_child1')]

##### In order to filter on a subtree, we can still use the prefix filter. For the value, we pass the path to any asset in the asset hierarchy. It may sound weird, but for a "list of things", the prefix filter just matches things from the start of the list.

**Note**: The returned subtree includes the asset located at the given path.

In [21]:
# 2) List all assets in part of the subtree
subtree_filter = Prefix(
    # We point the filter to the path property. This can also be done manually:
    # ("cdf_cdm", "CogniteAsset/v1", "path")
    property=asset_view.as_property_ref("path"),
    value=child1_asset.path,
)
subtree = client.data_modeling.instances.list(
    sources=asset_view,
    filter=subtree_filter,
    limit=None,
)
subtree

,space,external_id,version,last_updated_time,created_time,instance_type,path,root,parent,pathLastUpdatedTime,name
0,playground,delete_me_child1,2,2026-02-23 23:36:13.503,2026-02-23 23:36:12.857,node,"[{'space': 'playground', 'externalId': 'delete...","{'space': 'playground', 'externalId': 'delete_...","{'space': 'playground', 'externalId': 'delete_...",2026-02-23T23:36:13.503554+00:00,child1
1,playground,delete_me_child_of_child1,2,2026-02-23 23:36:13.503,2026-02-23 23:36:12.857,node,"[{'space': 'playground', 'externalId': 'delete...","{'space': 'playground', 'externalId': 'delete_...","{'space': 'playground', 'externalId': 'delete_...",2026-02-23T23:36:13.503554+00:00,child_of_child1


##### You can also go into the CDF user interface (fusion.cognite.com, org=`cognite-learn`, `ds-basics project`) and look at it there. If you do, go to Industrial tools worksapce --> Search: in the filtering you can apply Space=playground. Also changing to "Tree view" might make it easier for you to see and navigate the hierachy created.

### CogniteTimeSeries

#### Create time series node in CDM

In [26]:
ts = CogniteTimeSeriesApply(
    space="playground",
    external_id= f"{prefix}_timeseries",
    name=f"{prefix}_timeseries",
    time_series_type="numeric",
    is_step=False,
    # Optionally link to an asset:
    assets=[DirectRelationReference("playground", f"{prefix}_asset1")],
)
client.data_modeling.instances.apply(ts)

InstancesApplyResult(nodes=[<NodeApplyResult(space='playground', external_id='delete_me_timeseries', version=1) at 0x112b7f0e0>], edges=[])

#### Insert- and retrieve datapoints

In [27]:
from datetime import datetime, UTC

ts_id = NodeId("playground", f"{prefix}_timeseries")

datapoints = [
    (datetime(2026, 1, 1, tzinfo=UTC), 1234),
    (datetime(2026, 1, 2, tzinfo=UTC), 5678),
]
client.time_series.data.insert(
    instance_id=ts_id,
    datapoints=datapoints,
)

# Retrieve datapoints:
client.time_series.data.retrieve(
    instance_id=ts_id,
    start=datetime(2025, 1, 1, tzinfo=UTC),
    # end defaults to "now"
)

identifier,"NodeId(space='playground', external_id='delete_me_timeseries')"
2026-01-01,1234.0
2026-01-02,5678.0


## Updates in the Core Data Model (CDM)

You can also update nodes via `instances.apply` (re‑applying with changed properties), or use typed *Apply classes. Think of apply like upsert.

##### Lets change the names of the two first CogniteAssets created, and add a description.

**Note:** Please change the description string to something custom. This makes it easier to search for your CogniteAssets later.

In [28]:
asset_updates = [
    CogniteAssetApply(
        space="playground",
        external_id=f"{prefix}_asset1",
        # Here we add the prefix also to the name:
        name=f"{prefix}_asset1",
        # CHANGE THIS:
        description="Change the description to more easily recognize your own CogniteAsset",
    ),
    CogniteAssetApply(
        space="playground",
        external_id=f"{prefix}_asset2",
        name=f"{prefix}_asset2",  # Same change here.
        # CHANGE THIS:
        description="Change the description to more easily recognize your own CogniteAsset",
    ),
]
client.data_modeling.instances.apply(nodes=asset_updates)

InstancesApplyResult(nodes=[<NodeApplyResult(space='playground', external_id='delete_me_asset1', version=3) at 0x11291fd40>, <NodeApplyResult(space='playground', external_id='delete_me_asset2', version=3) at 0x11291f7d0>], edges=[])

##### Now, go to fusion.cognite.com, Industrial tools --> Search, and see if you can find your updated CogniteAssets!